<a href="https://colab.research.google.com/github/RoetGer/decisions-under-uncertainty/blob/main/data_science_and_stochastic_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cvxpy
!pip install cvxstoc

     |████████████████████████████████| 389kB 4.5MB/s 
  Created wheel for pymc: filename=pymc-2.3.8-cp37-cp37m-linux_x86_64.whl size=1352878 sha256=096f5934f670733b3422009de16e49cdcd5105a676d183f579a329ae53734eaf
  Stored in directory: /root/.cache/pip/wheels/0b/a8/e7/8f3ba91a39294d538a92db052fd1fcba1fca74a58c8b022026
Successfully built pymc


# Data Science and Stochastic Programming

In this notebook we explore, how stochastic programming can be used to incorporate uncertainty stemming from data science models into our decision making process.

In [2]:
from cvxstoc import NormalRandomVariable, expectation, prob
from cvxpy import Maximize, Problem
from cvxpy.expressions.variable import Variable
import numpy as np

# Samples to be taken
num_samples = 100

# Create problem data.
n = 10
mu = np.zeros(n)
Sigma = 0.1*np.eye(n)
p = NormalRandomVariable(mu, Sigma)
alpha = -1
beta = 0.05

# Create and solve stochastic optimization problem.
x = Variable(n)
p = Problem(
    Maximize(expectation(x.T*p, num_samples=num_samples)),
    [
      x >= 0, 
      x.T*np.ones(n) == 1,
      prob(x.T*p <= alpha, num_samples=num_samples) <= beta
    ]
)

p.solve()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


0.07730511150342165

In [3]:
?expectation

In [6]:
from cvxstoc import RandomVariable, RandomVariableFactory

In [5]:
??cvxstoc

In [4]:
??RandomVariable

In [21]:
??NormalRandomVariable

In [18]:
??RandomVariableFactory

In [9]:
import cvxstoc

In [20]:
??RandomVariableFactory.create_normal_rv

In [14]:
import pymc

In [24]:
??pymc.Normal

In [15]:
pymc.Normal(name="blub", mu=2., tau=1., size=(3,))

<pymc.distributions.new_dist_class.<locals>.new_class 'blub' at 0x7f7c9249aad0>

In [16]:
??RandomVariable

In [ ]:
def EmpiricalRandomVariable(samples):
    return create_empirical_rv(samples)

def create_empirical_rv(samples):
    rv_name = "empiricial_placeholder"

    metadata = {
        "mu": np.mean(samples)
    }

    


In [22]:
dir(pymc)

['AdaptiveMetropolis',
 'ArrayContainer',
 'Bernoulli',
 'Beta',
 'Betabin',
 'BinaryMetropolis',
 'Binomial',
 'Categorical',
 'Cauchy',
 'Chi2',
 'CircVonMises',
 'CircularStochastic',
 'CommonDeterministics',
 'CompletedDirichlet',
 'Container',
 'ContainerBase',
 'ContainerMeta',
 'ContainerRegistry',
 'Container_values',
 'Degenerate',
 'Deterministic',
 'DeterministicBase',
 'DeterministicMeta',
 'DeterministicRegistry',
 'DictContainer',
 'Dirichlet',
 'DiscreteMetropolis',
 'DiscreteUniform',
 'DiscreteUninformative',
 'DrawFromPrior',
 'Exponential',
 'Exponweib',
 'GOFpoints',
 'Gamma',
 'Geometric',
 'Gibbs',
 'HalfCauchy',
 'HalfNormal',
 'Histogram',
 'Hypergeometric',
 'Impute',
 'ImputeMissing',
 'Index',
 'InstantiationDecorators',
 'InvLogit',
 'InverseGamma',
 'Lambda',
 'Laplace',
 'LazyFunction',
 'LinearCombination',
 'ListContainer',
 'Logistic',
 'Logit',
 'Lognormal',
 'MAP',
 'MCMC',
 'Matplot',
 'Metropolis',
 'Model',
 'Multinomial',
 'MultivariateHypergeomet